In [39]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import librosa
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [41]:
import feature_engineering
from config import *

In [43]:
dataset_folder = '20210518-200601-KABES Sample Pack 1'
drums_df = pd.read_pickle(PICKLE_DATASETS_PATH / dataset_folder / DATASET_FILENAME)
drums_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   audio_path     42 non-null     object 
 1   file_stem      42 non-null     object 
 2   class          42 non-null     object 
 3   start_time     42 non-null     float64
 4   end_time       0 non-null      object 
 5   orig_duration  42 non-null     float64
 6   new_duration   42 non-null     float64
dtypes: float64(3), object(4)
memory usage: 2.6+ KB


In [44]:
drums_df.assign(test=None)

,audio_path,file_stem,class,start_time,end_time,orig_duration,new_duration,test
0,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 6,hat,0.0,None,1.000000,1.000000,None
1,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 7,hat,0.0,None,1.519002,1.519002,None
2,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 5,hat,0.0,None,0.391429,0.391429,None
3,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 4,hat,0.0,None,0.375011,0.375011,None
4,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 hh 9,hat,0.0,None,0.111973,0.111973,None
5,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 hh 8,hat,0.0,None,0.223900,0.223900,None
6,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 1,hat,0.0,None,1.000000,1.000000,None
7,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 3,hat,0.0,None,0.550023,0.550023,None
8,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 oh 2,hat,0.0,None,0.400000,0.400000,None
9,/Users/rayandaod/Documents/Prod/My_samples/KAB...,kbs1 hh 10,hat,0.0,None,0.223900,0.223900,None


In [ ]:
drums_df = feature_engineering.extract_all(dataset_folder=dataset_folder)
drums_df.head()

In [ ]:
df = df.assign(drum_type_labels=drum_type_labels)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
train_clips_df, val_clips_df = train_test_split(df, random_state=0)

In [ ]:
train_df = train_clips_df.drop(labels='drum_type_labels', axis=1)
train_df = train_df.drop(labels='drum_type', axis=1)
train_np = train_df.to_numpy()

In [ ]:
test_df = val_clips_df.drop(labels='drum_type_labels', axis=1)
test_df = test_df.drop(labels='drum_type', axis=1)
test_np = test_df.to_numpy()